In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from itertools import combinations
import matplotlib.pyplot as plt

import warnings
import tqdm
from warnings import simplefilter
from typing import List
import joblib
import os
import gc


warnings.filterwarnings('ignore')
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
train = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')

In [3]:
#整体特征
median_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()
std_sizes = train.groupby('stock_id')['bid_size'].std() + train.groupby('stock_id')['ask_size'].std()
max_sizes = train.groupby('stock_id')['bid_size'].max() + train.groupby('stock_id')['ask_size'].max()
min_sizes = train.groupby('stock_id')['bid_size'].min() + train.groupby('stock_id')['ask_size'].min()
mean_sizes = train.groupby('stock_id')['bid_size'].mean() + train.groupby('stock_id')['ask_size'].mean()
first_sizes = train.groupby('stock_id')['bid_size'].first() + train.groupby('stock_id')['ask_size'].first()
last_sizes = train.groupby('stock_id')['bid_size'].last() + train.groupby('stock_id')['ask_size'].last()
#可以再做日期的（好像没看到drop掉日期列）

train = train.dropna(subset=['target'])

def feature_eng(df):
    cols = [c for c in df.columns if c not in ['row_id', 'date_id','time_id']]
    df = df[cols]
    
    #匹配失败数量和匹配成功数量的比率
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    #供需市场的差额
    df['bid_ask_volume_diff'] = df['ask_size'] - df['bid_size']
    #供需市场总和
    df['bid_plus_ask_sizes'] = df['bid_size'] + df['ask_size']
    
    #供需价格的均值
    df['mid_price'] = (df['ask_price'] + df['bid_price']) / 2
    
    #整体数据情况
    df['median_size'] = df['stock_id'].map(median_sizes.to_dict())
    df['std_size'] = df['stock_id'].map(std_sizes.to_dict())
    df['max_size'] = df['stock_id'].map(max_sizes.to_dict())
    df['min_size'] = df['stock_id'].map(min_sizes.to_dict())
    df['mean_size'] = df['stock_id'].map(mean_sizes.to_dict())
    df['first_size'] = df['stock_id'].map(first_sizes.to_dict())    
    df['last_size'] = df['stock_id'].map(last_sizes.to_dict())       
    
    #整体市场规模和当前的市场规模比较
    df['high_volume'] = np.where(df['bid_plus_ask_sizes'] > df['median_size'], 1, 0)
    
    prices = ['reference_price', 'far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    #价格之间做差，做差/求和
    for c in combinations(prices, 2):
        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]} - {c[1]})/({c[0]} + {c[1]})')
        
    for c in combinations(prices, 3):
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1) - min_ - max_
        
        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_-mid_)/(mid_-min_ + 1e-4)
    
    gc.collect()
    
    return df

y = train['target'].values
X = feature_eng(train.drop(columns='target'))

y_min = np.min(y)
y_max = np.max(y)

In [4]:
def feat_eng_nn(df: pd.DataFrame):
    # change seconds_in_bucket to 9 categories (9 min) & make a new col
    df["stage"] = np.where(df["seconds_in_bucket"] > 300, 1, 0)
    df["min_in_bucket"] = df["seconds_in_bucket"]
    for i in range(9):
        t1, t2 = i * 60, ((i+1) * 60 if i < 8 else 541 )
        df.loc[(df["min_in_bucket"] >= t1) & (df["min_in_bucket"] < t2), "min_in_bucket"] = i 

    # create discrete feature
    int_feat = df.dtypes[df.dtypes == "int64"].to_dict().keys()
    
    # handle invaild values
    X_dsc = df[int_feat]
    for f in int_feat:
        mv = np.min(X_dsc[f])
        if mv < 0:
            X_dsc[f] += 0 - mv
    X_dsc = X_dsc.drop(columns="seconds_in_bucket")
    assert not X_dsc.isnull().any().any()
    cat_num = X_dsc.nunique()
    
    X_ctg = df.drop(columns=int_feat)
    X_ctg = X_ctg.fillna(0)
    
    
    return X_ctg, X_dsc, cat_num

X_ctg, X_dsc, cat_num = feat_eng_nn(X)
print(X_ctg.dtypes)
print()
print(X_dsc.dtypes)
print(X_dsc.head())
print(X_dsc.min())
print(cat_num)

imbalance_size                         float64
reference_price                        float64
matched_size                           float64
far_price                              float64
near_price                             float64
                                        ...   
far_price_bid_price_wap_imb2           float64
near_price_ask_price_bid_price_imb2    float64
near_price_ask_price_wap_imb2          float64
near_price_bid_price_wap_imb2          float64
ask_price_bid_price_wap_imb2           float64
Length: 71, dtype: object

stock_id                   int64
imbalance_buy_sell_flag    int64
high_volume                int64
stage                      int64
min_in_bucket              int64
dtype: object
   stock_id  imbalance_buy_sell_flag  high_volume  stage  min_in_bucket
0         0                        2            1      0              0
1         1                        0            0      0              0
2         2                        0            1      0     

In [5]:
print(np.isinf(X_ctg).any().any())
print(X_ctg.max())
# X_ctg["far_price_bid_price_wap_imb2"].replace(np.inf, 0)
print(np.max(X_ctg["far_price_bid_price_wap_imb2"].replace(np.inf, 0)))

False
imbalance_size                         2.982028e+09
reference_price                        1.077488e+00
matched_size                           7.713682e+09
far_price                              4.379531e+02
near_price                             1.309732e+00
                                           ...     
far_price_bid_price_wap_imb2           2.229371e+06
near_price_ask_price_bid_price_imb2    6.537748e+02
near_price_ask_price_wap_imb2          2.478137e+03
near_price_bid_price_wap_imb2          1.004060e+03
ask_price_bid_price_wap_imb2           1.090000e+02
Length: 71, dtype: float64
2229370.8366136355


In [6]:
class NNDataset(Dataset):
    def __init__(self, X_c, X_d, y_nn):
        self.X_c = X_c
        self.X_d = X_d
        self.y_nn = y_nn
    
    def __len__(self):
        return len(self.X_d)
    
    def __getitem__(self, idx):
        if self.y_nn is None:
            return torch.tensor(self.X_c.iloc[idx]).float(), torch.tensor(self.X_d.iloc[idx]).long()
        return torch.tensor(self.X_c.iloc[idx]).float(), torch.tensor(self.X_d.iloc[idx]).long(), torch.tensor(self.y_nn[idx]).float()

In [7]:
class CNN(nn.Module):
    def __init__(self,
                 num_features: int,
                 hidden_size: int,
                 n_categories: List[int],
                 emb_dim: int = 10,
                 dropout_cat: float = 0.2,
                 channel_1: int = 256,
                 channel_2: int = 512,
                 channel_3: int = 512,
                 dropout_top: float = 0.1,
                 dropout_mid: float = 0.3,
                 dropout_bottom: float = 0.2,
                 weight_norm: bool = True,
                 two_stage: bool = True,
                 celu: bool = True,
                 kernel1: int = 5,
                 leaky_relu: bool = False):
        super().__init__()

        num_targets = 1

        cha_1_reshape = int(hidden_size / channel_1)
        cha_po_1 = int(hidden_size / channel_1 / 2)
        cha_po_2 = int(hidden_size / channel_1 / 2 / 2) * channel_3

        self.cat_dim = emb_dim * len(n_categories)
        self.cha_1 = channel_1
        self.cha_2 = channel_2
        self.cha_3 = channel_3
        self.cha_1_reshape = cha_1_reshape
        self.cha_po_1 = cha_po_1
        self.cha_po_2 = cha_po_2
        self.two_stage = two_stage

        self.expand = nn.Sequential(
            nn.BatchNorm1d(num_features + self.cat_dim),
            nn.Dropout(dropout_top),
            nn.utils.weight_norm(nn.Linear(num_features + self.cat_dim, hidden_size), dim=None),
            nn.CELU(0.06) if celu else nn.ReLU()
        )

        def _norm(layer, dim=None):
            return nn.utils.weight_norm(layer, dim=dim) if weight_norm else layer

        self.conv1 = nn.Sequential(
            nn.BatchNorm1d(channel_1),
            nn.Dropout(dropout_top),
            _norm(nn.Conv1d(channel_1, channel_2, kernel_size=kernel1, stride=1, padding=kernel1 // 2, bias=False)),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(output_size=cha_po_1),
            nn.BatchNorm1d(channel_2),
            nn.Dropout(dropout_top),
            _norm(nn.Conv1d(channel_2, channel_2, kernel_size=3, stride=1, padding=1, bias=True)),
            nn.ReLU()
        )

        if self.two_stage:
            self.conv2 = nn.Sequential(
                nn.BatchNorm1d(channel_2),
                nn.Dropout(dropout_mid),
                _norm(nn.Conv1d(channel_2, channel_2, kernel_size=3, stride=1, padding=1, bias=True)),
                nn.ReLU(),
                nn.BatchNorm1d(channel_2),
                nn.Dropout(dropout_bottom),
                _norm(nn.Conv1d(channel_2, channel_3, kernel_size=5, stride=1, padding=2, bias=True)),
                nn.ReLU()
            )

        self.max_po_c2 = nn.MaxPool1d(kernel_size=4, stride=2, padding=1)

        self.flt = nn.Flatten()

        if leaky_relu:
            self.dense = nn.Sequential(
                nn.BatchNorm1d(cha_po_2),
                nn.Dropout(dropout_bottom),
                _norm(nn.Linear(cha_po_2, num_targets), dim=0),
                nn.LeakyReLU()
            )
        else:
            self.dense = nn.Sequential(
                nn.BatchNorm1d(cha_po_2),
                nn.Dropout(dropout_bottom),
                _norm(nn.Linear(cha_po_2, num_targets), dim=0)
            )

        self.embs = nn.ModuleList([nn.Embedding(x, emb_dim) for x in n_categories])
        self.cat_dim = emb_dim * len(n_categories)
        self.dropout_cat = nn.Dropout(dropout_cat)

    def forward(self, x_num, x_cat):
        embs = [embedding(x_cat[:, i]) for i, embedding in enumerate(self.embs)]
        x_cat_emb = self.dropout_cat(torch.cat(embs, 1))
        x = torch.cat([x_num, x_cat_emb], 1)

        x = self.expand(x)

        x = x.reshape(x.shape[0], self.cha_1, self.cha_1_reshape)

        x = self.conv1(x)

        if self.two_stage:
            x = self.conv2(x) * x

        x = self.max_po_c2(x)
        x = self.flt(x)
        x = self.dense(x)

        return torch.squeeze(x)



In [8]:
N_Folds = 4
kf = KFold(n_splits=N_Folds, shuffle=True, random_state=2023)
is_train = True
params_nn = {
    "batch_size": 1280,
    "lr": 1e-3,
    "epochs": 35,
    "val_iter": 1500,
    "train_log_step": 500,
    "scheduler_factor":0.25,
    "scd_patience": 2,
    "patience": 3
}

N, D_c = X_ctg.shape
print(N, D_c)
embed_dims = list(cat_num.to_dict().values())
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"device: {device}")
output_dir = "cnn_models"
os.system(f'mkdir {output_dir}')
if is_train:
    for fold_i, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
        # data
        tr_X_ctg, tr_X_dsc, tr_y = X_ctg.iloc[train_idx], X_dsc.iloc[train_idx], y[train_idx]
        valid_idx_small = valid_idx[:600000]
        val_X_ctg, val_X_dsc, val_y = X_ctg.iloc[valid_idx_small], X_dsc.iloc[valid_idx_small], y[valid_idx_small]
        
        # build torch dataset and dataloader 
        dataset_tr = NNDataset(tr_X_ctg, tr_X_dsc, tr_y)
        dataset_val = NNDataset(val_X_ctg, val_X_dsc, val_y)
        
        loader_tr = DataLoader(dataset_tr, batch_size=params_nn["batch_size"], shuffle=True, num_workers=16)
        loader_val = DataLoader(dataset_val, batch_size=params_nn["batch_size"], shuffle=False, num_workers=16)
        
        # build model and related modules
        model = CNN(num_features=D_c,
                    n_categories=embed_dims,
                    hidden_size=8*128,
                    emb_dim=16,
                    dropout_cat=0,
                    channel_1=128,
                    channel_2=3*128,
                    channel_3=3*128,
                    dropout_top=0,
                    dropout_mid=0,
                    dropout_bottom=0,
                    weight_norm=True,
                    two_stage=False,
                    celu=False,
                    kernel1=5,
                    leaky_relu=False)
        model.to(device)
        criterion = nn.L1Loss()
        optimizer = optim.Adam(model.parameters(), lr=params_nn["lr"])
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", 
                                                         factor=params_nn["scheduler_factor"],
                                                         patience=params_nn["scd_patience"])
        
        # begin training
        n_iter = np.ceil(len(dataset_tr) / params_nn["batch_size"])
        best_loss = np.inf
        last_epoch_loss = np.inf
        p = 0
        for epoch in range(params_nn["epochs"]):
            loss_epoch = []
            for i, (x_c, x_d, y_tr) in enumerate(loader_tr):
                optimizer.zero_grad()
                x_c, x_d, y_tr = x_c.to(device), x_d.to(device), y_tr.to(device)
                y_hat = model(x_c, x_d)
                loss = criterion(y_hat, y_tr)
                loss.backward()
                optimizer.step()
                
                loss_epoch.append(loss.detach().item())
                
                # log
                if i != 0 and i % params_nn["train_log_step"] == 0:
                    print(f"Fold {fold_i:1}, "
                          f"Epoch {epoch:2}/{params_nn['epochs']:2}, "
                          f"iter {i:5}/{n_iter:5}, "
                          f"loss {np.mean(loss_epoch[-params_nn['train_log_step']:]):8.4f}")
                
                # validate
                if i % params_nn["val_iter"] == 0 and i != 0 or i == n_iter - 1:
                    loss_val = []
#                     bar = tqdm.tqdm(total=np.ceil(len(dataset_val) / params_nn["batch_size"]))
                    with torch.no_grad():
                        for i, (x_c, x_d, y_val) in enumerate(loader_val):
                            x_c, x_d, y_val = x_c.to(device), x_d.to(device), y_val.to(device)

                            y_hat = model(x_c, x_d)
                            loss = criterion(y_hat, y_val)

                            loss_val.append(loss.item())
                            bar.update()
                    
                    loss_val_avg = np.mean(loss_val)
                    print(f"==> Val current best loss {best_loss:8.4f}")
                    print(f"Fold {fold_i:1}, "
                          f"Epoch {epoch:2}/{params_nn['epochs']:2}, "
                          f"Valid loss {loss_val_avg:8.4f}")
                    if loss_val_avg < best_loss:
                        best_loss = loss_val_avg
                        print(f"Loss decreases! current best loss {best_loss:8.4f}")
                        torch.save(model, os.path.join(output_dir, f"fold{fold_i}.pt"))
                    print()
            
            # early stop
            if loss_val_avg >= last_epoch_loss:
                p += 1
                print(f"Best loss doesn't decrease in this epoch, patience {p}/{params_nn['patience']}")
                if p >= params_nn["patience"]:
                    print(f"Reach patience, quit training of fold {fold_i}")
                    print()
                    break
            else:
                print(f"Epoch loss decreases from {last_epoch_loss:8.4f} to {loss_val_avg:8.4f}")
                last_epoch_loss = loss_val_avg
                p = 0
            
            # scheduler
            scheduler.step(loss_val_avg)
            print()

5237892 71
device: cpu


In [9]:
def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)
    step = np.sum(prices)/np.sum(std_error)
    out = prices-std_error*step
    
    return out

is_infer = False
N_Folds = 4
if is_infer:
    import optiver2023
    env = optiver2023.make_env()
    iter_test = env.iter_test()
    counter = 0
    predictions = []
    
    batch_size_test = 128
    for (test, revealed_targets, sample_prediction) in iter_test:
        feat = feature_eng(test)
        X_ctg, X_dsc, cat_num = feat_eng_nn(feat)
        test_dataset = NNDataset(X_ctg, X_dsc, None)
        test_loader = DataLoader(test_dataset,batch_size=batch_size_test,shuffle=False)
        
        fold_prediction = np.zeros((test.shape[0],))
        for fold in range(0, N_Folds):
            model_filename = f"/kaggle/input/cnn-optiver/cnn_models/fold{fold}.pt"
            m = torch.load(model_filename, map_location="cpu")
            for data_i, (x_c, x_d) in enumerate(test_loader):
                y_hat = m(x_c, x_d)
                fold_prediction[data_i * batch_size_test: (data_i + 1) * batch_size_test] += y_hat.detach().cpu().numpy()
                
        fold_prediction /= N_Folds
        fold_prediction = zero_sum(fold_prediction, test.loc[:,'bid_size'] + test.loc[:,'ask_size'])
        clipped_predictions = np.clip(fold_prediction, y_min, y_max)
        sample_prediction['target'] = clipped_predictions
        env.predict(sample_prediction)
        counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
